In [63]:
%matplotlib inline
import matplotlib.pyplot as plt

# Standard library packages
import os

# Import Numpy, Pandas and Seaborn
import numpy as np
import pandas as pd
import seaborn as sns

# Import biopython tools for running local BLASTX
from Bio.Blast.Applications import NcbiblastpCommandline

# Colour scale transformation
from matplotlib.colors import LogNorm

In [64]:
# Define input and output directories
with open("speciesList.txt") as f:
        speciesList = [x.rstrip() for x in f]
speciesStr = '_'.join(speciesList)
rPath = os.getcwd()
s1 = rPath+'/../../../../species/'+speciesList[0]+'/raw/longest_isoform_'+speciesList[0]+'Proteins.fasta'
s2 = rPath+'/../../../../species/'+speciesList[1]+'/raw/'+speciesList[1]+'Proteins.fasta'
outdir = rPath+'/../../../../species/crossSpecies/'+speciesStr+'/RBH/'
os.makedirs(outdir, exist_ok=True)
# Define output BLAST results
fwd_out = os.path.join(outdir, '05-fwd-results.tab')
rev_out = os.path.join(outdir, '05-rev-results.tab')

In [65]:
# Create BLAST command-lines for forward and reverse BLAST searches
blastp='/Users/mmeynadier/Documents/PhD/scripts/tools/ncbi-blast/bin/blastp'
fwd_blastp = NcbiblastpCommandline(cmd=blastp,query=s1, subject=s2, out=fwd_out,
                                   outfmt="6 qseqid sseqid pident qcovs qlen slen length bitscore evalue",
                                   max_target_seqs=1)
rev_blastp = NcbiblastpCommandline(cmd=blastp,query=s2, subject=s1, out=rev_out,
                                   outfmt="6 qseqid sseqid pident qcovs qlen slen length bitscore evalue",
                                   max_target_seqs=1)

# Inspect command-lines
print("FORWARD: %s" % fwd_blastp)
print("REVERSE: %s" % rev_blastp)

FORWARD: /Users/mmeynadier/Documents/PhD/scripts/tools/ncbi-blast/bin/blastp -out /Users/mmeynadier/Documents/PhD/scripts/GD_ECT_Cnidaria/py_scripts/miscellaneous/../../../../species/crossSpecies/Hydra_Hydractinia/RBH/05-fwd-results.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query /Users/mmeynadier/Documents/PhD/scripts/GD_ECT_Cnidaria/py_scripts/miscellaneous/../../../../species/Hydra/raw/longest_isoform_HydraProteins.fasta -max_target_seqs 1 -subject /Users/mmeynadier/Documents/PhD/scripts/GD_ECT_Cnidaria/py_scripts/miscellaneous/../../../../species/Hydractinia/raw/HydractiniaProteins.fasta
REVERSE: /Users/mmeynadier/Documents/PhD/scripts/tools/ncbi-blast/bin/blastp -out /Users/mmeynadier/Documents/PhD/scripts/GD_ECT_Cnidaria/py_scripts/miscellaneous/../../../../species/crossSpecies/Hydra_Hydractinia/RBH/05-rev-results.tab -outfmt "6 qseqid sseqid pident qcovs qlen slen length bitscore evalue" -query /Users/mmeynadier/Documents/PhD/scripts/GD_ECT_Cni

In [66]:
# Run BLAST searches
fwd_stdout, fwd_stderr = fwd_blastp()
rev_stdout, rev_stderr = rev_blastp()

# Check STDOUT, STDERR
print("FWD STDOUT: %s" % fwd_stdout)
print("FWD STDERR: %s" % fwd_stderr)
print("REV STDOUT: %s" % rev_stdout)
print("REV STDERR: %s" % rev_stderr)

In [ ]:
# Load the BLAST results into Pandas dataframes
fwd_results = pd.read_csv(fwd_out, sep="\t", header=None)
rev_results = pd.read_csv(rev_out, sep="\t", header=None)

# Add headers to forward and reverse results dataframes
headers = ["query", "subject", "identity", "coverage",
           "qlength", "slength", "alength",
           "bitscore", "E-value"]
fwd_results.columns = headers
rev_results.columns = headers

In [ ]:
# Create a new column in both dataframes: normalised bitscore
fwd_results['norm_bitscore'] = fwd_results.bitscore/fwd_results.qlength
rev_results['norm_bitscore'] = rev_results.bitscore/rev_results.qlength

# Create query and subject coverage columns in both dataframes
fwd_results['qcov'] = fwd_results.alength/fwd_results.qlength
rev_results['qcov'] = rev_results.alength/rev_results.qlength
fwd_results['scov'] = fwd_results.alength/fwd_results.slength
rev_results['scov'] = rev_results.alength/rev_results.slength

# Clip maximum coverage values at 1.0
fwd_results['qcov'] = fwd_results['qcov'].clip(upper=1)
rev_results['qcov'] = rev_results['qcov'].clip(upper=1)
fwd_results['scov'] = fwd_results['scov'].clip(upper=1)
rev_results['scov'] = rev_results['scov'].clip(upper=1)

In [ ]:
# Inspect the forward results data
fwd_results.head()

In [ ]:
# Inspect the reverse results data
rev_results.head()

In [ ]:
# Set up the figure
f, axes = plt.subplots(1, 2, figsize=(14, 7), sharex=True)
sns.despine(left=True)

# Plot distribution of forward and reverse hit bitscores
sns.distplot(fwd_results.norm_bitscore, color="b", ax=axes[0], axlabel="forward normalised bitscores")
sns.distplot(rev_results.norm_bitscore, color="g", ax=axes[1], axlabel="reverse normalised bitscores")

In [ ]:
# Plot 2D density histograms

# Calculate 2D density histograms for counts of matches at several coverage levels
(Hfwd, xedgesf, yedgesf) = np.histogram2d(fwd_results.qcov, fwd_results.scov, bins=20)
(Hrev, xedgesr, yedgesr) = np.histogram2d(rev_results.qcov, rev_results.scov, bins=20)

# Create a 1x2 figure array
fig, axes = plt.subplots(1, 2, figsize=(15, 6), sharex=True, sharey=True)

# Plot histogram for forward matches
im = axes[0].imshow(Hfwd, cmap=plt.cm.Blues, norm=LogNorm(),
                    extent=[xedgesf[0], xedgesf[-1], yedgesf[0], yedgesf[-1]],
                    origin='lower', aspect=1)
axes[0].set_title("Forward")
axes[0].set_xlabel("query")
axes[0].set_ylabel("subject")

# Plot histogram for reverse matches
im = axes[1].imshow(Hrev, cmap=plt.cm.Blues, norm=LogNorm(),
                    extent=[xedgesr[0], xedgesr[-1], yedgesr[0], yedgesr[-1]],
                    origin='lower', aspect=1)
axes[1].set_title("Reverse")
axes[1].set_xlabel("query")
axes[1].set_ylabel("subject")

# Add colourbars
fig.colorbar(im, ax=axes[0])
fig.colorbar(im, ax=axes[1])

In [ ]:
# Merge forward and reverse results
rbbh = pd.merge(fwd_results, rev_results[['query', 'subject']],
                left_on='subject', right_on='query',
                how='outer')

# Discard rows that are not RBH
rbbh = rbbh.loc[rbbh.query_x == rbbh.subject_y]

# Group duplicate RBH rows, taking the maximum value in each column
rbbh = rbbh.groupby(['query_x', 'subject_x']).max()

In [ ]:
# Inspect the results
rbbh.head()

In [ ]:
# Plot distribution of RBH bitscores
sns.distplot(rbbh.norm_bitscore, color="b", axlabel="RBH normalised bitscores")

In [ ]:
# Plot 2D density histograms

# Calculate 2D density histograms for counts of matches at several coverage levels
(H, xedges, yedges) = np.histogram2d(rbbh.qcov, rbbh.scov, bins=20)

# Create a 1x2 figure array
fig, ax = plt.subplots(1, 1, figsize=(6, 6), sharex=True, sharey=True)

# Plot histogram for RBBH
im = ax.imshow(H, cmap=plt.cm.Blues, norm=LogNorm(),
                 extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]],
                 origin='lower', aspect=1)
ax.set_title("RBBH")
ax.set_xlabel("query")
ax.set_ylabel("subject")

# Add colourbar
fig.colorbar(im, ax=ax)

In [ ]:
# Search for markers genes
"""
with open("cnidocyteMarkers.txt") as f:
        listMarkers = [x.rstrip() for x in f]
markerDf = pd.DataFrame()
query_x_list = []
subject_x_list = []
for i in listMarkers:
        res = rbbh[rbbh['query_y']==i] 
        if res.empty != True:
                query_x_list.append(res['subject_y'].iloc[0])
                subject_x_list.append(res['query_y'].iloc[0]) 
                markerDf = pd.concat([markerDf,res])
idx=0
markerDf.insert(loc=idx,column='subject_x',value=subject_x_list)
markerDf.insert(loc=idx,column='query_x',value=query_x_list)
#markerDf.to_csv(outdir+'/RBH_markerGenes.csv',index=False)
"""

In [ ]:
# Discard duplicate RBH

def discardDuplicate(filePath):
    df = pd.read_csv(filePath,sep='\t',header=None)
    df.columns = ['qseqid','sseqid','pident','qcovs','qlen','slen','length','bitscore','evalue']
    df2 = df.groupby(['qseqid', 'sseqid']).first()
    return df2

forwardCured = discardDuplicate(rPath+'/../../../../species/crossSpecies/'+speciesStr+'/RBH/05-rev-results.tab') 
reverseCured = discardDuplicate(rPath+'/../../../../species/crossSpecies/'+speciesStr+'/RBH/05-fwd-results.tab')

forwardCured.to_csv(rPath+'/../../../../species/crossSpecies/'+speciesStr+'/RBH/forwardRBH.txt',sep='\t')
reverseCured.to_csv(rPath+'/../../../../species/crossSpecies/'+speciesStr+'/RBH/backwardRBH.txt',sep='\t')

In [ ]:
# RBH over more than 2 species

"""
# Define input protein sequence files
infiles = ['']

# List to hold commands
my_commands = []

# Loop over input files and create forward and reverse commands
for idx, org1 in enumerate(infiles):
    for org2 in infiles[idx+1:]:
        fwd_out = "{0}_vs_{1}".format(os.path.split(org1)[-1],
                                      os.path.split(org2)[-1])      # Forward output file
        rev_out = "{1}_vs_{0}".format(os.path.split(org1)[-1],
                                      os.path.split(org2)[-1])      # Reverse output file
        fwd_blastp = NcbiblastpCommandline(cmd= blastp, query=org1, subject=org2, out=fwd_out,
                                   outfmt="\'6 qseqid sseqid pident qcovs qlen slen length bitscore evalue\'",
                                   max_target_seqs=1)  # Forward command
        rev_blastp = NcbiblastpCommandline(cmd=blastp, query=org2, subject=org1, out=rev_out,
                                   outfmt="\'6 qseqid sseqid pident qcovs qlen slen length bitscore evalue\'",
                                   max_target_seqs=1)  # Reverse commmand
        my_commands += [fwd_blastp, rev_blastp]
        
# Print out commands
print('\n\n'.join([str(cmd) for cmd in my_commands]))
"""

In [ ]:
"""
RBHpath = rPath + '/../../../../../species/crossSpecies/'+speciesList[0]+'_'+speciesList[1]+'/RBH/'
forward = pd.read_csv(RBHpath+speciesList[0]+'_'+speciesList[1]+'.txt',sep="\t")
backward = pd.read_csv(RBHpath+speciesList[1]+'_'+speciesList[0]+'.txt',sep="\t") 
backward[['qseqid','sseqid']] = backward[['sseqid','qseqid']] 
RBHdf = pd.merge(forward,backward,on=['qseqid','sseqid'],how='inner')
RBHdf.to_csv(RBHpath+'RBHmerged.txt',sep='\t')
"""